# 🚀 PirateNet on Google Colab
## 稀疏感測點重建通道流（JHTDB Re_tau=1000）

### 📋 本 Notebook 內容
1. 環境設定與依賴安裝
2. Google Drive 掛載（保存檢查點）
3. 資料下載（JHTDB 感測點）
4. 訓練 PirateNet（1000 epochs）
5. 評估與視覺化

### ⚙️ 配置特點
- ✅ **修復壁面邊界**：y ∈ [-1, 1]（原錯誤：[0, 2]）
- ✅ **修復學習率調度器**：明確啟用 warmup + exponential decay
- ✅ **GradNorm 自適應權重**：每 100 steps 更新
- ✅ **因果權重**：epsilon=1.0

### 🎯 預期結果
- 訓練時間：1-2 小時（T4/V100 GPU）
- 目標 L2 誤差：≤ 15%
- 檢查點自動保存到 Google Drive

## 1️⃣ 檢查 GPU 可用性

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU 不可用！請到 Runtime > Change runtime type > GPU")

## 2️⃣ 掛載 Google Drive（保存檢查點）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 創建專案目錄
!mkdir -p /content/drive/MyDrive/pinns-mvp/checkpoints
!mkdir -p /content/drive/MyDrive/pinns-mvp/results
print("✅ Google Drive 已掛載")

## 3️⃣ 克隆專案並安裝依賴

In [ ]:
# 克隆專案（請替換為您的 GitHub URL）
!git clone https://github.com/your-username/pinns-mvp.git
%cd pinns-mvp

# 或者從 Google Drive 載入（如果已上傳）
# %cd /content/drive/MyDrive/pinns-mvp

In [ ]:
# 安裝依賴
!pip install -q pyyaml h5py tensorboard matplotlib seaborn scipy
!pip install -q pyJHTDB  # JHTDB 官方客戶端（可能需要身份驗證）

print("✅ 依賴已安裝")

## 4️⃣ 下載 JHTDB 資料

⚠️ **重要**：如果本地已有資料，可跳過此步驟並從 Google Drive 複製

In [ ]:
# 選項 A：從 JHTDB 下載（需要網路連接）
!python scripts/fetch_channel_flow.py --K 50 --output data/jhtdb/channel_flow_re1000

# 選項 B：從 Google Drive 複製已有資料
# !cp -r /content/drive/MyDrive/pinns-mvp/data ./

# 驗證資料完整性
!ls -lh data/jhtdb/channel_flow_re1000/

## 5️⃣ 驗證配置文件

In [ ]:
import yaml

# 載入配置
with open('configs/colab_piratenet_1k.yml', 'r') as f:
    config = yaml.safe_load(f)

# 顯示關鍵配置
print("📋 訓練配置摘要：")
print(f"實驗名稱: {config['experiment']['name']}")
print(f"設備: {config['experiment']['device']}")
print(f"模型: {config['model']['depth']}×{config['model']['width']}")
print(f"Epochs: {config['training']['epochs']}")
print(f"批次大小: {config['training']['batch_size']}")
print(f"感測點數: {config['sensors']['K']}")
print(f"\n✅ 關鍵修復：")
print(f"壁面位置: y ∈ [{config['data']['domain']['y_min']}, {config['data']['domain']['y_max']}]")
print(f"學習率調度器: {config['training']['scheduler']['type']}")
print(f"GradNorm: {config['losses']['adaptive_weights']['enabled']}")

## 6️⃣ 開始訓練 🚀

### ⚠️ 重要提示
- 訓練時間：約 1-2 小時（T4 GPU）
- 檢查點每 100 epochs 保存一次
- 可隨時中斷並從檢查點恢復

In [ ]:
# 首次訓練
!python scripts/train.py --cfg configs/colab_piratenet_1k.yml

# 從檢查點恢復（如果中斷）
# !python scripts/train.py --cfg configs/colab_piratenet_1k.yml \
#     --resume checkpoints/colab_piratenet_1k/latest.pth

## 7️⃣ 即時監控訓練（另開 Cell 運行）

In [ ]:
# 載入 TensorBoard
%load_ext tensorboard
%tensorboard --logdir checkpoints/colab_piratenet_1k

In [ ]:
# 或者查看訓練日誌（即時更新）
!tail -f log/colab_piratenet_1k/training_stdout.log

## 8️⃣ 評估訓練結果

In [ ]:
# 評估最終模型
!python scripts/evaluate_piratenet_vs_jhtdb.py \
    --checkpoint checkpoints/colab_piratenet_1k/epoch_1000.pth \
    --config configs/colab_piratenet_1k.yml \
    --device cuda

## 9️⃣ 視覺化結果

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json

# 載入評估結果
with open('results/colab_piratenet_1k/vs_jhtdb_statistics.json', 'r') as f:
    stats = json.load(f)

# 顯示誤差統計
print("📊 場變量誤差分析")
print("=" * 60)
for field, metrics in stats['statistics'].items():
    rel_l2_pct = metrics['relative_l2'] * 100
    print(f"{field}: {rel_l2_pct:.2f}% (RMSE: {metrics['rmse']:.4e})")

print("\n🎯 成功標準檢驗")
print("=" * 60)
criteria = stats['success_criteria']
print(f"速度場: {'✅ 通過' if criteria['velocity_success'] else '❌ 未通過'}")
print(f"壓力場: {'✅ 通過' if criteria['pressure_success'] else '❌ 未通過'}")
print(f"整體: {'🎉 成功' if criteria['overall_success'] else '⚠️ 需要改進'}")

In [ ]:
# 繪製誤差柱狀圖
fields = list(stats['statistics'].keys())
errors = [stats['statistics'][f]['relative_l2'] * 100 for f in fields]

plt.figure(figsize=(10, 6))
plt.bar(fields, errors, alpha=0.7)
plt.axhline(y=15, color='r', linestyle='--', label='目標: 15%')
plt.xlabel('場變量', fontsize=12)
plt.ylabel('相對 L2 誤差 (%)', fontsize=12)
plt.title('PirateNet 預測誤差分析', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('results/colab_piratenet_1k/error_comparison.png', dpi=150)
plt.show()

In [ ]:
# 繪製場分佈（中心平面）
predictions = np.load('results/colab_piratenet_1k/vs_jhtdb_predictions.npz')
grid_shape = predictions['grid_shape']

# 重塑為 3D 網格
u_pred = predictions['u_pred'].reshape(grid_shape)
u_true = predictions['u_true'].reshape(grid_shape)

# 取中心切面
mid_z = grid_shape[2] // 2
u_pred_slice = u_pred[:, :, mid_z]
u_true_slice = u_true[:, :, mid_z]

# 並排顯示
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

vmin, vmax = u_true_slice.min(), u_true_slice.max()

im1 = axes[0].imshow(u_true_slice.T, cmap='RdBu_r', vmin=vmin, vmax=vmax, aspect='auto')
axes[0].set_title('JHTDB 真實場 (u)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(u_pred_slice.T, cmap='RdBu_r', vmin=vmin, vmax=vmax, aspect='auto')
axes[1].set_title('PirateNet 預測場 (u)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
plt.colorbar(im2, ax=axes[1])

error = np.abs(u_pred_slice - u_true_slice)
im3 = axes[2].imshow(error.T, cmap='hot', aspect='auto')
axes[2].set_title('絕對誤差 |u_pred - u_true|', fontsize=12, fontweight='bold')
axes[2].set_xlabel('x')
axes[2].set_ylabel('y')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.savefig('results/colab_piratenet_1k/field_comparison.png', dpi=150)
plt.show()

## 🔟 保存結果到 Google Drive

In [ ]:
# 複製檢查點到 Google Drive
!cp -r checkpoints/colab_piratenet_1k /content/drive/MyDrive/pinns-mvp/checkpoints/

# 複製結果到 Google Drive
!cp -r results/colab_piratenet_1k /content/drive/MyDrive/pinns-mvp/results/

print("✅ 結果已保存到 Google Drive")
print("📂 位置: /content/drive/MyDrive/pinns-mvp/")

## 📋 下一步建議

### 如果結果未達標（L2 > 15%）
1. **增加訓練時間**：改為 2000-5000 epochs
2. **增加感測點**：K=50 → K=80-100
3. **調整網路結構**：width=768 → width=1024
4. **檢查壁面損失**：確認 `wall_loss` 不為 0

### 如果達標
1. **執行 K-scan 實驗**：驗證最少感測點數
2. **進行不確定性量化**：使用 Ensemble 訓練
3. **撰寫評估報告**

### 故障排除
- **NaN 損失**：降低學習率（1e-3 → 5e-4）
- **GPU 記憶體不足**：減少批次大小（8192 → 4096）
- **訓練中斷**：從最新檢查點恢復（見步驟 6）